In [1]:
import pkuseg
import jieba
import random
import pandas as pd
from tqdm import tqdm
random.seed(1)
conf_path = os.path.dirname(os.path.abspath(__file__))
work_path = os.path.dirname(os.path.dirname(conf_path))
base_dir = os.path.join(work_path, "data")
# base_dir = '../data/'

NameError: name 'os' is not defined

In [ ]:
def is_chinese(string):
    for uchar in string:
        if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
            return True
        else:
            return False

def filter_all(segs):
    segs = filter(lambda x:x not in stopwords, segs)  #去掉停用词
#     segs = filter(lambda x:len(x)>1,segs)
    segs = filter(lambda x:is_chinese(x)==True, segs)
    segs = filter(lambda x:x != '了' and x != '的', segs)
    string_list = []
    for i in segs:
        string_list.append(i)
    # TODO
    string = (' '.join(string_list))
    return string
        
# pku > jieba
def dump_file(df, filename, mode='train'):
    seg = pkuseg.pkuseg(model_name='web')
    with open(filename, 'w',encoding='utf-8') as f:
        for i in df.index.values:
            if random.random() > 0.5:
                segs = seg.cut(df.text[i])
            else:
                segs = jieba.cut(df.text[i])
            string = filter_all(segs)
            if mode == 'train' or mode == 'val':
                string = df.stance[i] + ' ' + string
                string = string.lstrip()
            f.write(string + '\n')

In [ ]:
df_source = pd.read_csv(base_dir+'train.csv',delimiter="\t")
df_extra = pd.read_csv(base_dir+'test_pred.csv',index_col=0)
df = pd.concat([df_extra, df_source])
df.dropna(inplace=True)

mapping = {'FAVOR':'__label__A', 'AGAINST':'__label__B', 'NONE':'__label__C'}
df['stance'] = df['stance'].map(mapping)

df.drop_duplicates(subset='text', keep='first', inplace=True)
df.index = range(len(df))

stopwords = []
for line in open('stopwords.txt', encoding='utf-8'):
    stopwords.append(line)

In [ ]:
def process_extra_data(df_extra):
    df_a = df_extra[df.stance == '__label__A'].sample(frac=1)
    df_b = df_extra[df.stance == '__label__B'].sample(frac=1)
    df_c = df_extra[df.stance == '__label__C'].sample(frac=1)
    
    dfc = pd.DataFrame(columns=['target', 'text', 'stance'])
    
    for i in range(len(df_c)):
        if i != len(df_c) - 1:
            string_a = filter_all(jieba.cut(df_c.iloc[i].text))
            string_a_list = string_a.split(' ')
            string_a_list = random.sample(string_a_list, 2)
            string_b = filter_all(jieba.cut(df_c.iloc[i+1].text))
            string_b_list = string_b.split(' ')
            string_b_list = random.sample(string_b_list, 2)
            string_a_list.extend(string_b_list)
            
            data = {'target':['whatever'], 'text':[','.join(string_a_list)], 'stance':['__label__A']}
            df_tmp = pd.DataFrame(data)
            dfc = pd.concat([df_tmp,df_c])
    
    return dfc

In [ ]:
df_c = df.loc[df.stance == '__label__C'].sample(frac=0.4)
df_a = df.loc[df.stance == '__label__A'].sample(frac=0.04)
df_b = df.loc[df.stance == '__label__B'].sample(frac=0.04)

In [ ]:
df = df.sample(frac=1)
df['stance'].value_counts()

In [ ]:
train_len = int(len(df) * 0.75)
df_train = df.iloc[:train_len]
df_val = df.iloc[train_len:]

In [ ]:
dump_file(df_train, 'train.txt', 'train')
dump_file(df_val, 'val.txt', 'val')

In [ ]:
def read_data_file(path):
    seg = pkuseg.pkuseg(model_name='web')
    lines = open(path, 'r', encoding='utf-8').read().splitlines()
    x_list = []
    y_list = []
    for line in tqdm(lines):
    rows = line.split('\t')
    if len(rows) > 1:
        if random.random() > 0.5:
            segs = jieba.cut(rows[0])
        else:
            segs = seg.cut(rows[0])
    x_list.append(filter_all(segs))
    y_list.append(rows[1])
    return x_list, y_list

In [ ]:
train_x, train_y = read_data_file(base_dir+'train.txt')
val_x, val_y = read_data_file(base_dir+'val.txt')
test_x, test_y = read_data_file(base_dir+'test.txt')

In [ ]:
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.embeddings import WordEmbedding
# embedding = BERTEmbedding(base_dir+'chinese_L-12_H-768_A-12/', task=kashgari.CLASSIFICATION)
embedding = WordEmbedding(base_dir+'sgns.weibo.bigram-char', task=kashgari.CLASSIFICATION)
kashgari.config.use_cudnn_cell = True

In [ ]:
from kashgari.tasks.classification import BiLSTM_Model

hyper = BiLSTM_Model.get_default_hyper_parameters()
hyper['layer_bi_lstm']['units']=32
# hyper['spatial_dropout']['rate']=0.5
# hyper['dropout']['rate']=0.5
# hyper['dropout_rnn']['rate']=0.5
model = BiLSTM_Model(embedding, hyper_parameters=hyper)
model.fit(train_x, train_y, val_x, val_y, batch_size=32, epochs=20)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
test = pd.read_csv(base_dir+'test.csv', delimiter='\t')
labels = []
for line in tqdm(test.text.values):
    x_list = filter_all(jieba.cut(line, cut_all=True))
    labels.append(model.predict(x_list)[0])
        
test['idx'] = range(len(test))
test['stance'] = labels
test = test.drop(['target', 'text'], axis=1)

test.to_csv(base_dir+'test_pred.csv',index=False,header=False)

In [ ]:
BiLSTM_Model.get_default_hyper_parameters()
